In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import datetime
import pytz

In [ ]:
# Lendo dados coletados da API
df = pd.read_csv('modo_data.csv')

In [ ]:
def Hour_Diff(h1,h2):
    """
    Faz a diferença entre duas horas dadas e retorna em minutos
    
    Parameters
    ----------
    h1, h2 : unix timestamp
        Hora inicio e fim para ser feito o cálculo da diferença
        
    Returns
    ---------
    diff : float
        Diferença entre as duas horas dadas em minutos
        
    """
    
    h1Aux = datetime.datetime.fromtimestamp(h1)
    h2Aux = datetime.datetime.fromtimestamp(h2)
    diff = abs((h1Aux - h2Aux)).total_seconds()/60
    
    return diff

In [ ]:
def str_to_datetime(df_time):
    """ 
    Reformatando de string para datetime.
    
    Parameters
    ----------
    df_time : pandas.DataFrame, string
        Dataframe com strings a serem convertidas para datetime.
    
    Returns
    ----------
    date_list : pandas.DataFrame, datetime
        Dataframe com valores em datetime para possíveis fusos de Vancouver.
    
    """
    date_list = []
    
    # Formatos de fuso horário comum de Vancouver e 
    # fuso horário característico de horário de verão
    format_string = ['%Y-%m-%d %H:%M:%S.%f-08:00', '%Y-%m-%d %H:%M:%S.%f-07:00',
                     '%Y-%m-%d %H:%M:%S-08:00', '%Y-%m-%d %H:%M:%S-07:00']
    
    print('Begin: '+str(datetime.datetime.now()))
    
    for date in df_time:
        for fmt in format_string:
            try:
                date_list.append(datetime.datetime.strptime(str(date), fmt))
                break
            except:
                print('Error in : '+str(date))
                pass
            
    print('End: '+str(datetime.datetime.now()))
    return pd.DataFrame(date_list)

In [ ]:
def get_car_ids(car_list):
    """
    Coleta todos os IDs de carros coletados, sem repetições, de uma lista.
    
    Parameters
    -----------
    car_list : int list ou pandas.DataFrame
        Lista de todos os IDs coletados.
    
    Returns
    ----------
    car_ids : int
        Lista com todos os IDs já coletados, sem repetições.
        
    Notes
    ---------
    A coleta dos IDs é realizada de tal forma para obter IDs de veículos que 
    por utilização da API podem não ser retornados, como os que estão em manutenção
    não estão na frota atual.
    """
    
    car_ids = []

    for car in car_list:
        if (car in car_ids):
            continue
        else:
            car_ids.append(car)
    
    return car_ids

In [ ]:
# Convertendo datetime strings para o tipo datetime
df['Capture_time'] = str_to_datetime(df['Capture_time'])

In [ ]:
# Coletando todos os IDs dos veículos
car_ids = get_car_ids(df['CarID'])

In [ ]:
new_travel = []
cancel = []
parked = []

for j in range(len(car_ids)):
    carID = car_ids[j]
    carDF = []
    
    carDF = df[df['CarID'] == carID]
    # Eliminando intervalos de disponibilidade futuros
    carDF.drop_duplicates(subset='Capture_time', keep='first', inplace=True)
    carDF = carDF.sort_values(by=['Capture_time'])
    carDF.index = range(len(carDF))
    
    i = 1
    
    while i < len(carDF):
        try:
            # Extração com base no Start Time
            # Horarios estão no fuso horário de Vancouver
            start_time_atual = int(carDF['StartTime'].iloc[i])

            start_time_anterior = int(carDF['StartTime'].iloc[i-1])

            request_start_atual = carDF['RequestStart'].iloc[i]

            request_start_anterior = carDF['RequestStart'].iloc[i-1]

            capture_timestamp = int(carDF['Capture_time'].iloc[i].timestamp())

            # Verifica se a coleta começou com o carro parado
            if (i == 1 and start_time_atual == request_start_atual):

                start_parked = capture_timestamp                


            # Verifica se a coleta começou com o carro andando
            if (i == 1 and start_time_atual > request_start_atual): 

                # Registra uma nova viagem
                new_travel.append([carID, capture_timestamp, start_time_atual, True])

            # Se o inicio do tempo de disponibilidade > tempo de requisição E estava Parado temos uma nova viagem.  
            elif (start_time_atual > request_start_atual and start_time_anterior == request_start_anterior):

                # Registra uma nova viagem
                new_travel.append([carID, capture_timestamp, start_time_atual, True])

                # Registra o final do tempo em que estava estacionado
                if (start_parked > 0):
                    parked.append([carID, start_parked, capture_timestamp])
                    start_parked = -1


            # Se o inicio do tempo de disponibilidade anterior < tempo de disponibilidade atual E 
            # está andando temos uma nova viagem/extensão.
            elif (start_time_anterior < start_time_atual and start_time_atual > request_start_atual):

                new_travel.append([carID, capture_timestamp, start_time_atual, False])             


            # Se estava andando e agora está parado.
            if (start_time_atual == request_start_atual and start_time_anterior > request_start_anterior):

                # Inicio do tempo estacionado
                start_parked = capture_timestamp

                if(new_travel != []):

                    # Se a diferença entre hora atual e o inicio da ultima viagem < 30 min => cancelamento
                    # Senão => Diminuição/Cancelamento
                    if (Hour_Diff(capture_timestamp, new_travel[-1][1]) < 30):

                        cancel.append([carID, capture_timestamp, start_time_anterior, True])

                    # Tolerância de 20 min para dizer que foi uma diminuição ou 
                    # cancelamento em vez de um termino de viagem
                    elif(Hour_Diff(capture_timestamp, new_travel[-1][2]) > 20):

                        cancel.append([carID, capture_timestamp, start_time_anterior, False])


            # Se acontecer um aumento do tempo de diponibilidade enquanto está andando, 
            # ocorreu um cancelamento ou diminuição
            if (start_time_atual > request_start_atual and start_time_anterior > start_time_atual):

                cancel.append([carID, capture_timestamp, start_time_anterior, False])             

            
            i += 1
                
                    
        except Exception as e:
            # Exception será gerada nos casos em que o carro não está disponível
            print(e)
            continue

dfTravels = pd.DataFrame(new_travel,columns=['car_id', 'start', 'end', 'only_new_reserves'])
dfCancel = pd.DataFrame(cancel, columns=['car_id', 'capture_time', 'previous_start', 'only_cancel'])
dfParked = pd.DataFrame(parked, columns=['car_id', 'start', 'end'])

In [ ]:
dfTravels.to_csv('travels.csv', index=False, encoding='utf-8')
dfCancel.to_csv('cancel.csv', index=False, encoding='utf-8')
dfParked.to_csv('parked.csv', index=False, encoding='utf-8')